Import packages

In [23]:
import numpy as np
import pandas as pd
import os
import glob

Data import

In [24]:
df = pd.read_excel("benchmarks.xlsx", sheet_name="data")

Data analysis

In [28]:
df.head(5)

,Currency,Name,RIC,06.10.2023,05.10.2023,04.10.2023,03.10.2023,02.10.2023,01.10.2023,30.09.2023,...,10.01.2021,09.01.2021,08.01.2021,07.01.2021,06.01.2021,05.01.2021,04.01.2021,03.01.2021,02.01.2021,01.01.2021
0,AUD,AUD ON deposit,AUDOND=,2.57,2.57,2.57,2.57,2.57,2.57,2.57,...,-0.17,-0.17,-0.17,-0.17,-0.17,-0.17,-0.17,-0.17,-0.17,-0.17
1,BRL,Selic ON,BRSELICD=CBBR,8.78,8.78,8.78,8.78,8.78,8.78,8.78,...,1.32,1.32,1.32,1.32,1.32,1.32,1.32,NaN,NaN,NaN
2,CHF,SARON,SARON.S,1.18,1.18,1.18,1.18,1.18,1.19,1.19,...,-0.50,-0.50,-0.50,-0.50,-0.50,-0.50,-0.50,NaN,NaN,NaN
3,CNY,CNY ON deposit,CNYONID=CNR,1.23,1.23,1.23,1.23,1.23,1.23,1.23,...,0.82,0.82,0.82,0.80,0.49,0.46,1.32,6.94,6.94,6.94
4,EUR,ESTER,EUROSTR=,2.70,2.71,2.71,2.71,2.70,2.69,2.69,...,-0.39,-0.39,-0.39,-0.39,-0.39,-0.39,-0.39,NaN,NaN,NaN


In [29]:
df.tail(5)

,Currency,Name,RIC,06.10.2023,05.10.2023,04.10.2023,03.10.2023,02.10.2023,01.10.2023,30.09.2023,...,10.01.2021,09.01.2021,08.01.2021,07.01.2021,06.01.2021,05.01.2021,04.01.2021,03.01.2021,02.01.2021,01.01.2021
29,SEK,SWESTR,SWESTR=RIKS,2.70,2.70,2.70,2.70,2.70,2.70,2.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,SGD,SORA,SORA=MAST,2.70,2.52,2.49,2.49,2.54,2.65,2.65,...,0.14,0.14,0.14,0.12,0.09,0.08,0.06,NaN,NaN,NaN
31,THB,THOR,THONRP=BKTH,1.73,1.73,1.73,1.72,1.73,1.73,1.73,...,0.34,0.34,0.34,0.34,0.34,0.34,0.34,NaN,NaN,NaN
32,UAH,UONIA,UONIA=NBUK,11.10,11.10,11.10,11.10,11.10,10.99,10.99,...,3.49,3.49,3.49,3.49,3.49,3.51,3.47,NaN,NaN,NaN
33,ZAR,ZAR ON deposit,ZAROND=,5.76,3.12,3.12,4.16,3.95,4.16,4.16,...,2.74,2.74,2.74,2.57,3.30,3.47,4.34,3.47,3.47,3.47


In [30]:
df.shape

(34, 1012)

Data modelling

In [31]:
# Delete column 'Name'
df = df.drop("Name", axis=1)

In [32]:
# Unpivot all columns except 'Currency', 'RIC' -> melt columns containing date
df = pd.melt(df, id_vars=['Currency', 'RIC'], var_name='Date', value_name='Universal Close Price')

In [33]:
# Add empty column 'Universal Bid Price'
df['Universal Bid Price'] = np.nan

In [34]:
# Reorder columns
df = df[['Date', 'RIC', 'Currency', 'Universal Close Price', 'Universal Bid Price']]

# Rename columns
datascope_colnames = {'Date': 'Trade Date',
                      'Currency': 'Currency Code'}
df = df.rename(columns=datascope_colnames)

Create file per date

In [35]:
# Unique dates
date_sequence = df['Trade Date'].unique()
print('There are', len(date_sequence), 'date stamps')

There are 1009 date stamps


In [36]:
def get_filename(date):
    """
    Generate a filename based on the provided date.

    Args:
        date (str): A date string in the format "DD.MM.YYYY"

    Returns:
        str: A filename in the format "YYYYMMDD_on_ir_benchmark".

    Example:
        >>> get_filename('30.09.2023')
        '20230930_on_ir_benchmark'
    """
    date_list = date.split('.')[::-1]
    str_empty = ''
    prefix_date = str_empty.join(date_list)
    filename = f"{prefix_date}_on_ir_benchmark"
    return filename

In [37]:
# for loop to create an overnight benchmark file for each date
for date_item in date_sequence:
    # Get date
    df_subset = df[df['Trade Date'] == date_item]
    # Get file name & set path
    file_name = get_filename(date_item)
    file_name_path = f"files_created/{file_name}.csv"
    # Create a file
    df_subset.to_csv(file_name_path, index=None, sep=',', decimal=',')

Test: Show created file

In [38]:
df_test = pd.read_csv('files_created/20230930_on_ir_benchmark.csv')

In [39]:
df_test.head(5)

,Trade Date,RIC,Currency Code,Universal Close Price,Universal Bid Price
0,30.09.2023,AUDOND=,AUD,"2,57",NaN
1,30.09.2023,BRSELICD=CBBR,BRL,"8,78",NaN
2,30.09.2023,SARON.S,CHF,"1,19",NaN
3,30.09.2023,CNYONID=CNR,CNY,"1,23",NaN
4,30.09.2023,EUROSTR=,EUR,"2,69",NaN


Test: Merge all files

In [40]:
# merging the files 
file_format = os.path.join('files_created', '*benchmark*') 
  
# A list of all joined files is returned 
files_list = glob.glob(file_format) 

In [41]:
df_concat = pd.concat([pd.read_csv(file, decimal=',') for file in files_list], ignore_index=True)

In [43]:
df_concat.head()

,Trade Date,RIC,Currency Code,Universal Close Price,Universal Bid Price
0,01.01.2021,AUDOND=,AUD,-0.13481,NaN
1,01.01.2021,BRSELICD=CBBR,BRL,NaN,NaN
2,01.01.2021,SARON.S,CHF,NaN,NaN
3,01.01.2021,CNYONID=CNR,CNY,5.50342,NaN
4,01.01.2021,EUROSTR=,EUR,NaN,NaN


In [44]:
df_concat.tail()

,Trade Date,RIC,Currency Code,Universal Close Price,Universal Bid Price
35093,27.10.2023,THONRP=BKTH,THB,1.976854,NaN
35094,26.10.2023,UONIA=NBUK,NaN,12.688000,NaN
35095,27.10.2023,ZAROND=,ZAR,5.551000,1.983800
35096,27.10.2023,NOKOND=,NOK,3.290950,1.176110
35097,27.10.2023,CNYONID=CNR,NaN,1.378234,0.492549


In [45]:
df_concat.shape

(35098, 5)